In [1]:
import pandas as pd
import numpy as np
from bridge_specs import box
from Cross import calc_centroid as cg


In [2]:
def loader(unit_wt, length, width, height, type):
    if type == 'r':
        load = unit_wt*length*width*height
    elif type == 't':
        load = unit_wt*length*width*height*0.5
    
    return load
def z(base, clr, width, x, type):
    """base = width of base along with taper
       clr = overhang part from where centroid is calculated
        width = width of object whose lever arm is required
        object = shape of object
    """
    y = []
    if type == 'r':
        y.append(cg('rectangle',[width,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't1':
        y.append(cg('triangle_2',[width,0],[x,0]))
        return (-(clr+base/2)+y[0][0])
    y = []
    if type == 't2':
        y.append(cg('triangle_1',[width,0],[x,0]))
        return (-(clr+base/2)+y[0][0])

Abutment details

In [3]:
bearing_ht = 0.3 #assumed including pedestal
abut = {
    'total_ht' : 7.0,
    'wc' : 0.075,
    'backw_ht' : box['pillar_hei']+bearing_ht+0.075, #including wearing cource
    'cap':0.5,
    'stem_ht': 7-box['pillar_hei']+bearing_ht+0.075-0.5,
    'bw_width':0.3,
    'edge_to_bearing_c':0.6,
    'bw_to_bearing_c': 0.6,
    'cap_width':0.3+0.6+0.6,
    'stem_width':1.25,
    'be_taperwidth':0.10,
    'width_of_abut':8.4,
    'ttl_stembase_width':1.25+0.10,
    'approach_slab_dep':0.3,
    'approach_slab_len':3.5,
    'haunch':{'stem_haunch':[0.2,0.25],'approach_slab_haunch':[[0.25,0.25],[0.25,0.25]]} #list of width and height.
}

Live Loads, Dead loads and Pedestrian loads

In [4]:
#dummy loads
dl = 3000 #dead loads
ll = 1000 #live loads including impact
pdl = 200 #pedestrian loads

Material properties

In [5]:
fck = 30 #grade of concrete = M30
fy = 500 #grade of steel = Fe500
unitwt_conc = 25 #unit weight of concrete = 25KN/m^3
unitwt_bfill = 20 #unit weight of backfill = 20KN/m^3

Self weight loads of abutment

In [14]:
# comp = ['stem_rect','stem_tri','cap','backwall','stem_haunch','approach_slab_haunch']
loads = [loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['stem_width'],'r'),
         loader(unitwt_conc,abut['stem_ht'],abut['width_of_abut'],abut['be_taperwidth'],'t'),
         loader(unitwt_conc,abut['cap'],abut['width_of_abut'],abut['cap_width'],'r'),
         loader(unitwt_conc,abut['backw_ht'],abut['width_of_abut'],abut['bw_width'],'r'),
         loader(unitwt_conc,abut['haunch']['stem_haunch'][0],abut['width_of_abut'],abut['haunch']['stem_haunch'][1],'r'),
         [loader(unitwt_conc,abut['haunch']['approach_slab_haunch'][0][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch'][0][1],'r'),loader(unitwt_conc,abut['haunch']['approach_slab_haunch'][1][0],abut['width_of_abut'],abut['haunch']['approach_slab_haunch'][1][1],'t')]]
lever_arm = [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['stem_width'],abut['haunch']['stem_haunch'][0],'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['be_taperwidth'],abut['haunch']['stem_haunch'][0]+abut['stem_width'],'t1'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['cap_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['bw_width'],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],abut['haunch']['stem_haunch'][0],0,'t2'),5),
            [round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch'][0][0],0,'r'),5),
            round(z(abut['ttl_stembase_width'],abut['haunch']['stem_haunch'][0],-abut['haunch']['approach_slab_haunch'][0][1],0,'r'),5)]]
print(loads)
print(lever_arm)
moments = []
for index,i in enumerate(loads):
    if isinstance(i,float)==True:
        moments.append(loads[index]*lever_arm[index])
    elif isinstance(i,list)==True:
        temp=[]
        for j in range(len(i)):
            temp.append(loads[index][j]*lever_arm[index][j])
        moments.append(temp)
A1 = pd.DataFrame({"Load":loads,"Lever arm":lever_arm,"Moment":moments},index = ['Stem(rectangle)','Stem(triangle)','Cap','Backwall','Stem haunch','Approach slab haunch'])
print(A1)
            
    
    

[964.6875, 38.587500000000006, 157.5, 225.225, 10.5, [13.125, 6.5625]]
[-0.05, 0.60833, -0.125, -0.725, -0.74167, [-1.0, -1.0]]
                                  Load     Lever arm              Moment
Stem(rectangle)               964.6875         -0.05          -48.234375
Stem(triangle)                 38.5875       0.60833           23.473934
Cap                              157.5        -0.125            -19.6875
Backwall                       225.225        -0.725         -163.288125
Stem haunch                       10.5      -0.74167           -7.787535
Approach slab haunch  [13.125, 6.5625]  [-1.0, -1.0]  [-13.125, -6.5625]
